## Introduction

In this guide, we will walk you through building a Retrieval Augmented Generation (RAG) application using Couchbase Capella as the database, [gpt-4o](https://platform.openai.com/docs/models/gpt-4o) model as the large language model provided by OpenAI. We will use the [text-embedding-3-large](https://platform.openai.com/docs/guides/embeddings/embedding-models) model for generating embeddings.

This notebook demonstrates how to build a RAG system using:
- The [BBC News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime) containing news articles
- Couchbase Capella as the vector store with Hyperscale and Composite Vector Indexes for vector search
- LlamaIndex framework for the RAG pipeline
- OpenAI for embeddings and text generation

Couchbase offers two types of vector indexes for high-performance vector search:

**Hyperscale Vector Indexes**: Best for pure vector searches - content discovery, recommendations, semantic search. Designed to scale to billions of vectors with low memory footprint and optimized for concurrent operations.

**Composite Vector Indexes**: Best for filtered vector searches - combines vector search with scalar value filtering. Efficient pre-filtering allows scalar attributes to reduce the vector comparison scope.

For more details, see the [Couchbase Vector Index documentation](https://docs.couchbase.com/cloud/vector-index/use-vector-indexes.html).

Semantic search goes beyond simple keyword matching by understanding the context and meaning behind the words in a query, making it an essential tool for applications that require intelligent information retrieval. This tutorial will equip you with the knowledge to create a fully functional RAG system using OpenAI Services and LlamaIndex with Couchbase's Hyperscale and Composite Vector Indexes.

## Before you start

### Create and Deploy Your Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy an operational cluster.

To know more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html). 

### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met:

* Have a multi-node Capella cluster running the Data, Query, Index, and Search services.
* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.

### OpenAI Models Setup

In order to create the RAG application, we need an embedding model to ingest the documents for Vector Search and a large language model (LLM) for generating the responses based on the context. 

For this implementation, we'll use OpenAI's models which provide state-of-the-art performance for both embeddings and text generation:

**Embedding Model**: We'll use OpenAI's `text-embedding-3-large` model, which provides high-quality embeddings with 3,072 dimensions for semantic search capabilities.

**Large Language Model**: We'll use OpenAI's `gpt-4o` model for generating responses based on the retrieved context. This model offers excellent reasoning capabilities and can handle complex queries effectively.

**Prerequisites for OpenAI Integration**:
* Create an OpenAI account at [platform.openai.com](https://platform.openai.com)
* Generate an API key from your OpenAI dashboard
* Ensure you have sufficient credits or a valid payment method set up
* Set up your API key as an environment variable or input it securely in the notebook

For more details about OpenAI's models and pricing, please refer to the [OpenAI documentation](https://platform.openai.com/docs/models).


## Installing Necessary Libraries
To build our RAG system, we need a set of libraries. The libraries we install handle everything from connecting to databases to performing AI tasks. Each library has a specific role: Couchbase libraries manage database operations, LlamaIndex handles AI model integrations, and we will use the OpenAI SDK for generating embeddings and calling OpenAI's language models.


In [1]:
# Install required packages
%pip install --no-user --quiet datasets==3.6.0 llama-index==0.14.13 llama-index-vector-stores-couchbase==0.6.0 llama-index-embeddings-openai==0.5.1 llama-index-llms-openai==0.6.18 python-dotenv==1.2.1


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.


In [2]:
import getpass
import hashlib
import json
import logging
import os
import sys
import time
from datetime import timedelta

from dotenv import load_dotenv

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import CouchbaseException
from couchbase.management.buckets import CreateBucketSettings
from couchbase.options import ClusterOptions, KnownConfigProfiles, QueryOptions

from datasets import load_dataset

from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode
from llama_index.vector_stores.couchbase import CouchbaseQueryVectorStore, QueryVectorSearchSimilarity, QueryVectorSearchType
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

/Users/kaustavghosh/Desktop/vector-search-cookbook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Sensitive Information
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, collection names, and API keys. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script also validates that all required inputs are provided, raising an error if any crucial information is missing. This approach ensures that your integration is both secure and correctly configured without hardcoding sensitive information, enhancing the overall security and maintainability of your code.

**OPENAI_API_KEY** is your OpenAI API key which can be obtained from your OpenAI dashboard at [platform.openai.com](https://platform.openai.com/api-keys).

**INDEX_NAME** is the name of the vector index we will create for vector search operations using Hyperscale or Composite Vector Indexes.

In [3]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass.getpass('Enter your OpenAI API key: ')
CB_HOST = os.getenv('CB_HOST', 'couchbase://localhost') or input('Enter Couchbase host (default: couchbase://localhost): ') or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME', 'Administrator') or input('Enter Couchbase username (default: Administrator): ') or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD', 'password') or getpass.getpass('Enter Couchbase password (default: password): ') or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME', 'vector-search-testing') or input('Enter Couchbase bucket name: ')
SCOPE_NAME = os.getenv('SCOPE_NAME', 'shared') or input('Enter scope name: ')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'llamaindex') or input('Enter collection name: ')
INDEX_NAME = os.getenv('INDEX_NAME', 'vector_search_llamaindex') or input('Enter index name: ')

if not all([OPENAI_API_KEY, CB_HOST, CB_USERNAME, CB_PASSWORD, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME, INDEX_NAME]):
    raise ValueError("All configuration variables must be provided.")

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Setting Up Logging
Logging is essential for tracking the execution of our script and debugging any issues that may arise. We set up a logger that will display information about the script's progress, including timestamps and log levels.


In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logging.getLogger("httpx").setLevel(logging.WARNING)

## Connecting to Couchbase Capella
The next step is to establish a connection to our Couchbase Capella cluster. This connection will allow us to interact with the database, store and retrieve documents, and perform vector searches.


In [5]:
try:
    # Initialize the Couchbase Cluster
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    options.apply_profile(KnownConfigProfiles.WanDevelopment)
    # Connect to the cluster
    cluster = Cluster(CB_HOST, options)
    
    # Wait for the cluster to be ready
    cluster.wait_until_ready(timedelta(seconds=5))

    logging.info("Successfully connected to the Couchbase cluster")
except CouchbaseException as e:
    raise RuntimeError(f"Failed to connect to Couchbase: {str(e)}")

2026-02-10 14:45:22,139 - INFO - Successfully connected to the Couchbase cluster


## Setting Up the Bucket, Scope, and Collection
Before we can store our data, we need to ensure that the appropriate bucket, scope, and collection exist in our Couchbase cluster. The code below checks if these components exist and creates them if they don't, providing a foundation for storing our vector embeddings and documents.

In [6]:
# Create bucket if it does not exist
bucket_manager = cluster.buckets()
try:
    bucket_manager.get_bucket(CB_BUCKET_NAME)
    print(f"Bucket '{CB_BUCKET_NAME}' already exists.")
except Exception as e:
    print(f"Bucket '{CB_BUCKET_NAME}' does not exist. Creating bucket...")
    bucket_settings = CreateBucketSettings(name=CB_BUCKET_NAME, ram_quota_mb=500)
    bucket_manager.create_bucket(bucket_settings)
    print(f"Bucket '{CB_BUCKET_NAME}' created successfully.")

# Create scope and collection if they do not exist
collection_manager = cluster.bucket(CB_BUCKET_NAME).collections()
scopes = collection_manager.get_all_scopes()
scope_exists = any(scope.name == SCOPE_NAME for scope in scopes)

if scope_exists:
    print(f"Scope '{SCOPE_NAME}' already exists.")
else:
    print(f"Scope '{SCOPE_NAME}' does not exist. Creating scope...")
    collection_manager.create_scope(SCOPE_NAME)
    print(f"Scope '{SCOPE_NAME}' created successfully.")

collections = [collection.name for scope in scopes if scope.name == SCOPE_NAME for collection in scope.collections]
collection_exists = COLLECTION_NAME in collections

if collection_exists:
    print(f"Collection '{COLLECTION_NAME}' already exists in scope '{SCOPE_NAME}'.")
else:
    print(f"Collection '{COLLECTION_NAME}' does not exist in scope '{SCOPE_NAME}'. Creating collection...")
    collection_manager.create_collection(collection_name=COLLECTION_NAME, scope_name=SCOPE_NAME)
    print(f"Collection '{COLLECTION_NAME}' created successfully.")

scope = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME)

Bucket 'vector-search-testing' already exists.
Scope 'shared' already exists.
Collection 'llamaindex' already exists in scope 'shared'.


## Setting Up Query-Based Vector Search
In this section, we'll set up the Couchbase vector store using Couchbase Hyperscale and Composite Vector Index for high-performance vector search. Unlike FTS-based vector search, Hyperscale and Composite Vector Index search provides optimized performance for pure vector similarity operations and can scale to billions of vectors with low memory footprint.

Hyperscale and Composite Vector Index search supports two main index types:
- **Hyperscale Vector Indexes**: Best for pure vector searches with high performance and concurrent operations
- **Composite Vector Indexes**: Best for filtered vector searches combining vector similarity with scalar filtering

For this tutorial, we'll use the Query vector store.

## Load the BBC News Dataset
To build a RAG engine, we need data to search through. We use the [BBC Realtime News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime), a dataset with up-to-date BBC news articles grouped by month. This dataset contains articles that were created after the LLM was trained. It will showcase the use of RAG to augment the LLM. 

The BBC News dataset's varied content allows us to simulate real-world scenarios where users ask complex questions, enabling us to fine-tune our RAG's ability to understand and respond to various types of queries.


In [7]:
try:
    news_dataset = load_dataset('RealTimeData/bbc_news_alltime', '2024-12', split="train")
    print(f"Loaded the BBC News dataset with {len(news_dataset)} rows")
except Exception as e:
    raise ValueError(f"Error loading TREC dataset: {str(e)}")

Loaded the BBC News dataset with 2687 rows


## Preview the Data

In [8]:
# Print the first two examples from the dataset
print("Dataset columns:", news_dataset.column_names)
print("\nFirst two examples:")
print(news_dataset[:2])

Dataset columns: ['title', 'published_date', 'authors', 'description', 'section', 'content', 'link', 'top_image']

First two examples:
{'title': ["Pakistan protest: Bushra Bibi's march for Imran Khan disappeared - BBC News", 'Lockdown DIY linked to Walleys Quarry gases - BBC News'], 'published_date': ['2024-12-01', '2024-12-01'], 'authors': ['https://www.facebook.com/bbcnews', 'https://www.facebook.com/bbcnews'], 'description': ["Imran Khan's third wife guided protesters to the heart of the capital - and then disappeared.", 'An academic says an increase in plasterboard sent to landfill could be behind a spike in smells.'], 'section': ['Asia', 'Stoke & Staffordshire'], 'content': ['Bushra Bibi led a protest to free Imran Khan - what happened next is a mystery\n\nImran Khan\'s wife, Bushra Bibi, encouraged protesters into the heart of Pakistan\'s capital, Islamabad\n\nA charred lorry, empty tear gas shells and posters of former Pakistan Prime Minister Imran Khan - it was all that remaine

## Preparing the Data for RAG

We need to extract the context passages from the dataset to use as our knowledge base for the RAG system.

In [9]:
try:
    news_articles = news_dataset
    unique_articles = {}

    for article in news_articles:
        content = article.get("content")
        if content:
            content_hash = hashlib.md5(content.encode()).hexdigest()
            if content_hash not in unique_articles:
                unique_articles[content_hash] = article

    unique_news_articles = list(unique_articles.values())

    logging.info(f"We have {len(unique_news_articles)} unique articles in our database.")
except Exception as e:
    raise RuntimeError(f"Failed to prepare data: {str(e)}")

2026-02-10 14:45:26,118 - INFO - We have 1749 unique articles in our database.


## Creating Embeddings using OpenAI
Embeddings are numerical representations of text that capture semantic meaning. Unlike keyword-based search, embeddings enable semantic search to understand context and retrieve documents that are conceptually similar even without exact keyword matches. We'll use OpenAI's `text-embedding-3-large` model to create high-quality embeddings with 3,072 dimensions. This model transforms our text data into vector representations that can be efficiently searched, with a batch size of 30 for optimal processing.


In [10]:
try:
    # Set up the embedding model
    embed_model = OpenAIEmbedding(
        api_key=OPENAI_API_KEY,
        embed_batch_size=30,
        model="text-embedding-3-large"
    )
    
    # Configure LlamaIndex to use this embedding model
    Settings.embed_model = embed_model
    print("Successfully created embedding model")
except Exception as e:
    raise ValueError(f"Error creating embedding model: {str(e)}")

Successfully created embedding model


## Testing the Embeddings Model
We can test the embeddings model by generating an embedding for a string

In [11]:
try:
    test_embedding = embed_model.get_text_embedding("this is a test sentence")
    logging.info(f"Embedding dimension: {len(test_embedding)}")
except Exception as e:
    raise RuntimeError(f"Failed to generate test embedding: {str(e)}")

2026-02-10 14:45:28,056 - INFO - Embedding dimension: 3072


## Understanding Hyperscale and Composite Vector Search

### Optimizing Vector Search with Hyperscale and Composite Vector Index

With Couchbase 8.0+, you can leverage the power of query-based vector search, which offers significant performance improvements over traditional Full-Text Search (FTS) approaches for vector-first workloads. Hyperscale and Composite Vector Index search provides high-performance vector similarity search with advanced filtering capabilities and is designed to scale to billions of vectors.

#### Hyperscale/Composite vs Search Vector Index: Choosing the Right Approach

| Feature               | Hyperscale/Composite Vector Index                                               | Search Vector Index                         |
| --------------------- | --------------------------------------------------------------- | ----------------------------------------- |
| **Best For**          | Vector-first workloads, complex filtering, high QPS performance| Hybrid search and high recall rates      |
| **Couchbase Version** | 8.0.0+                                                         | 7.6+                                      |
| **Filtering**         | Pre-filtering with `WHERE` clauses (Composite) or post-filtering (Hyperscale) | Pre-filtering with flexible ordering |
| **Scalability**       | Up to billions of vectors (Hyperscale)                              | Up to 10 million vectors                  |
| **Performance**       | Optimized for concurrent operations with low memory footprint  | Good for mixed text and vector queries   |

#### Query-Based Vector Index Types

Couchbase offers two distinct query-based vector index types, each optimized for different use cases:

##### Hyperscale Vector Indexes

- **Best for**: Pure vector searches like content discovery, recommendations, and semantic search
- **Use when**: You primarily perform vector-only queries without complex scalar filtering
- **Features**: 
  - High performance with low memory footprint
  - Optimized for concurrent operations
  - Designed to scale to billions of vectors
  - Supports post-scan filtering for basic metadata filtering

##### Composite Vector Indexes

- **Best for**: Filtered vector searches that combine vector similarity with scalar value filtering
- **Use when**: Your queries combine vector similarity with scalar filters that eliminate large portions of data
- **Features**: 
  - Efficient pre-filtering where scalar attributes reduce the vector comparison scope
  - Best for well-defined workloads requiring complex filtering
  - Supports range lookups combined with vector search

#### Understanding Index Configuration

The `index_description` parameter controls how Couchbase optimizes vector storage through centroids and quantization.

##### Index Description Format: `'IVF[<centroids>],{PQ|SQ}<settings>'`

**Centroids (IVF - Inverted File)**:
- Controls how the dataset is subdivided for faster searches
- More centroids = faster search, slower training
- If omitted (like `IVF,SQ8`), Couchbase auto-selects based on dataset size

**Quantization Options**:
- **SQ (Scalar Quantization)**: SQ4, SQ6, SQ8 (4, 6, or 8 bits per dimension)
- **PQ (Product Quantization)**: PQ<subquantizers>x<bits> (e.g., PQ32x8)

For detailed configuration options, see the [Quantization & Centroid Settings](https://docs.couchbase.com/cloud/vector-index/hyperscale-vector-index.html#algo_settings).

## Setting Up the Couchbase Query Vector Store
The query vector store is set up to store the documents from the dataset using Couchbase's query-based vector search capabilities. This vector store is optimized for high-performance vector similarity search operations and can scale to billions of vectors.

In [12]:
try:
    # Create the Couchbase vector store
    vector_store = CouchbaseQueryVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        search_type=QueryVectorSearchType.ANN,
        similarity=QueryVectorSearchSimilarity.DOT,
        nprobes=10
    )
    print("Successfully created vector store")
except Exception as e:
    raise ValueError(f"Failed to create vector store: {str(e)}")

Successfully created vector store


## Creating LlamaIndex Documents
In this section, we'll process our news articles and create LlamaIndex Document objects.
Each Document is created with specific metadata and formatting templates to control what the LLM and embedding model see.
We'll observe examples of the formatted content to understand how the documents are structured.

In [13]:
llama_documents = []
# Process and store documents
for article in unique_news_articles:
    try:
        document = Document(
            text=article["content"],
            metadata={
                "title": article["title"],
                "description": article["description"],
                "published_date": article["published_date"],
                "link": article["link"],
            },
            excluded_llm_metadata_keys=["description"],
            excluded_embed_metadata_keys=["description", "published_date", "link"],
            metadata_template="{key}=>{value}",
            text_template="Metadata: \n{metadata_str}\n-----\nContent: {content}",
        )
        llama_documents.append(document)
    except Exception as e:
        print(f"Failed to save document to vector store: {str(e)}")
        continue

# Observing an example of what the LLM and Embedding model receive as input
print("The LLM sees this:")
print(llama_documents[0].get_content(metadata_mode=MetadataMode.LLM))
print("The Embedding model sees this:")
print(llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED))

The LLM sees this:
Metadata: 
title=>Pakistan protest: Bushra Bibi's march for Imran Khan disappeared - BBC News
published_date=>2024-12-01
link=>http://www.bbc.co.uk/news/articles/cvg02lvj1e7o
-----
Content: Bushra Bibi led a protest to free Imran Khan - what happened next is a mystery

Imran Khan's wife, Bushra Bibi, encouraged protesters into the heart of Pakistan's capital, Islamabad

A charred lorry, empty tear gas shells and posters of former Pakistan Prime Minister Imran Khan - it was all that remained of a massive protest led by Khan’s wife, Bushra Bibi, that had sent the entire capital into lockdown. Just a day earlier, faith healer Bibi - wrapped in a white shawl, her face covered by a white veil - stood atop a shipping container on the edge of the city as thousands of her husband’s devoted followers waved flags and chanted slogans beneath her. It was the latest protest to flare since Khan, the 72-year-old cricketing icon-turned-politician, was jailed more than a year ago aft

## Creating and Running the Ingestion Pipeline

In this section, we'll create an ingestion pipeline to process our documents. The pipeline will:

1. Split the documents into smaller chunks (nodes) using the SentenceSplitter
2. Generate embeddings for each node using our embedding model
3. Store these nodes with their embeddings in our Couchbase vector store

This process transforms our raw documents into a searchable knowledge base that can be queried semantically.

In [14]:
try:
    # Process documents: split into nodes, generate embeddings, and store in vector database
    index_pipeline = IngestionPipeline(
        transformations=[SentenceSplitter(), embed_model],
        vector_store=vector_store,
    )

    nodes = index_pipeline.run(documents=llama_documents)
    logging.info(f"Successfully ingested {len(nodes)} nodes into the vector store.")
except Exception as e:
    raise RuntimeError(f"Failed to run ingestion pipeline: {str(e)}")

2026-02-10 14:46:39,361 - INFO - Successfully ingested 2329 nodes into the vector store.


## Using OpenAI's Large Language Model (LLM)
Large language models are AI systems that are trained to understand and generate human language. We'll be using OpenAI's `gpt-4o` model to process user queries and generate meaningful responses based on the retrieved context from our Couchbase vector store. This model is a key component of our RAG system, allowing it to go beyond simple keyword matching and truly understand the intent behind a query. By integrating OpenAI's LLM, we equip our RAG system with the ability to interpret complex queries, understand the nuances of language, and provide more accurate and contextually relevant responses.

The language model's ability to understand context and generate coherent responses is what makes our RAG system truly intelligent. It can not only find the right information but also present it in a way that is useful and understandable to the user.

The LLM is configured using LlamaIndex's OpenAI-like provider with OpenAI's API endpoint and your OpenAI API key for seamless integration with their services.

In [15]:
try:
    # Set up the LLM
    llm = OpenAI(
        api_key=OPENAI_API_KEY,
        model="gpt-4o",
        
    )
    # Configure LlamaIndex to use this LLM
    Settings.llm = llm
    logging.info("Successfully created the OpenAI LLM")
except Exception as e:
    raise ValueError(f"Error creating OpenAI LLM: {str(e)}")

2026-02-10 14:46:39,369 - INFO - Successfully created the OpenAI LLM


## Creating the Vector Store Index

In this section, we'll create a VectorStoreIndex from our Couchbase vector store. This index serves as the foundation for our RAG system, enabling semantic search capabilities and efficient retrieval of relevant information.

The VectorStoreIndex provides a high-level interface to interact with our vector store, allowing us to:
1. Perform semantic searches based on user queries
2. Retrieve the most relevant documents or chunks
3. Generate contextually appropriate responses using our LLM


In [16]:
try:
    # Create your index
    index = VectorStoreIndex.from_vector_store(vector_store)
    rag = index.as_query_engine()
    logging.info("Successfully created vector store index and query engine.")
except Exception as e:
    raise RuntimeError(f"Failed to create vector store index: {str(e)}")

2026-02-10 14:46:39,377 - INFO - Successfully created vector store index and query engine.


## Understanding Semantic Search in Couchbase

Semantic search goes beyond traditional keyword matching by understanding the meaning and context behind queries. Here's how it works in Couchbase:

### How Semantic Search Works

1. **Vector Embeddings**: Documents and queries are converted into high-dimensional vectors using an embeddings model (in our case, OpenAI's text-embedding-3-large model)

2. **Similarity Calculation**: When a query is made, Couchbase compares the query vector against stored document vectors using the DOT product similarity metric

3. **Result Ranking**: Documents are ranked by their vector similarity scores

4. **Flexible Configuration**: Different similarity metrics (dot product, cosine, euclidean) and embedding models can be used based on your needs

Now let's see semantic search in action and measure its performance with different optimization strategies.

## Vector Search Performance Testing

### Phase 1: Baseline Performance (No Hyperscale Index)

First, we'll run a RAG query without using a Hyperscale index to establish our baseline performance. This search uses linear brute force which compares the query vector against every document in the collection. This works for small datasets but can become slow as the dataset grows.

In [17]:
# Sample query from the dataset

query = "Who will Daniel Dubois fight in Saudi Arabia on 22 February?"

try:
    # Perform the semantic search
    start_time = time.time()
    response = rag.query(query)
    search_elapsed_time = time.time() - start_time

    # Display search results
    print(f"\nSemantic Search Results (completed in {search_elapsed_time:.2f} seconds):")
    print(response)

except Exception as e:
    raise RuntimeError(f"Error performing semantic search: {e}")


Semantic Search Results (completed in 1.37 seconds):
Daniel Dubois will fight Joseph Parker in Saudi Arabia on 22 February.


### Creating the Hyperscale Index

Now we'll create a Hyperscale index to significantly improve query performance. The index uses IVF (Inverted File) with PQ (Product Quantization) for optimal balance between speed and accuracy.

The index configuration uses:
- **IVF1024**: 1024 centroids for fast search
- **PQ32x8**: Product quantization with 32 subquantizers and 8 bits each
- **DOT similarity**: Optimized for dot product similarity matching

In [18]:
# Create a Hyperscale Vector Index for optimized vector search
try:
    hyperscale_index_name = f"{INDEX_NAME}_hyperscale"

    options = {
        "dimension": 3072,
        "description": "IVF1024,PQ32x8",
        "similarity": "DOT",
    }
    scope.query(
        f"""
        CREATE INDEX {hyperscale_index_name}
        ON {COLLECTION_NAME} (embedding VECTOR)
        USING GSI WITH {json.dumps(options)}
        """,
    QueryOptions(
        timeout=timedelta(seconds=300)
    )).execute()
    print(f"Successfully created Hyperscale Vector Index: {hyperscale_index_name}")
except Exception as e:
    print(f"Hyperscale Vector Index may already exist or error occurred: {str(e)}")


Hyperscale Vector Index may already exist or error occurred: QueryIndexAlreadyExistsException(<ec=18, category=couchbase.common, message=index_exists (18), context=QueryErrorContext({'last_dispatched_to': '[::1]:8093', 'last_dispatched_from': '[::1]:50293', 'retry_attempts': 0, 'client_context_id': '307cb1-5404-c145-ee6c-386a11e6a2e210', 'method': 'POST', 'path': '/query/service', 'http_status': 409, 'http_body': '{\n"requestID": "7fbcc5a7-e359-4d59-b069-605cbbd0883f",\n"clientContextID": "307cb1-5404-c145-ee6c-386a11e6a2e210",\n"signature": null,\n"results": [\n],\n"errors": [{"code":4300,"msg":"The index vector_search_llamaindex_hyperscale already exists.","reason":{"name":"vector_search_llamaindex_hyperscale"}}],\n"status": "fatal"\n}\n', 'first_error_code': 4300, 'first_error_message': 'The index vector_search_llamaindex_hyperscale already exists.', 'statement': '\n        CREATE INDEX vector_search_llamaindex_hyperscale\n        ON llamaindex (embedding VECTOR)\n        USING GSI 

### Phase 2: Hyperscale-Optimized Performance

Now let's run the same RAG query using the Hyperscale index we just created. You'll notice improved performance as the index efficiently retrieves data.

In [19]:
# Test the optimized vector search with Hyperscale Vector Index
query = "Who will Daniel Dubois fight in Saudi Arabia on 22 February?"
try:
    # Create a new query engine using the optimized vector store
    optimized_rag = index.as_query_engine()
    
    # Perform the semantic search with Hyperscale Vector Index optimization
    start_time = time.time()
    response = optimized_rag.query(query)
    search_elapsed_time = time.time() - start_time

    # Display search results
    print(f"\nOptimized Vector Search Results (completed in {search_elapsed_time:.2f} seconds):")
    print(response)

except Exception as e:
    raise RuntimeError(f"Error performing optimized semantic search: {e}")



Optimized Vector Search Results (completed in 1.98 seconds):
Daniel Dubois will fight Joseph Parker in Saudi Arabia on 22 February.


### Performance Analysis Summary

The performance comparison between Phase 1 (baseline) and Phase 2 (Hyperscale-optimized) demonstrates the significant benefits of using Hyperscale indexes for vector search:

| Metric | Phase 1 (Baseline) | Phase 2 (Hyperscale) | Improvement |
|--------|-------------------|---------------------|-------------|
| Search Method | Brute force | IVF + PQ indexed | Optimized |
| Index Type | None | Hyperscale (IVF1024,PQ32x8) | Optimized |
| Scalability | Limited | Billions of vectors | High |

**Key Takeaways:**
- **Hyperscale indexes** provide substantial performance improvements for semantic search
- The **IVF + PQ configuration** offers an excellent balance of speed and accuracy
- For production workloads, always create appropriate indexes based on your query patterns
- Consider **Composite indexes** when you need to combine vector search with scalar filtering

## Conclusion

You've built a RAG system using Couchbase Hyperscale/Composite indexes with OpenAI and LlamaIndex. For the Search Vector Index alternative, see the [search_based tutorial](https://developer.couchbase.com/tutorial-openai-llamaindex-couchbase-rag-with-search-vector-index).